In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from scipy.stats import skewnorm

# 设置随机种子确保结果可复现
np.random.seed(42)

# 生成具有偏态分布的基础能力值 (更符合实际情况)
n_samples = 100
base_ability = skewnorm.rvs(a=-4, loc=0.6, scale=0.25, size=n_samples)
base_ability = np.clip(base_ability, 0, 1)

# 创建考试分数的相关性结构
correlation_matrix = np.array([[1.0, 0.7], [0.7, 1.0]])
chol = np.linalg.cholesky(correlation_matrix)

# 生成相关的随机噪声
random_noise = np.random.normal(0, 0.15, size=(n_samples, 2))
correlated_noise = np.dot(random_noise, chol)

# 结合基础能力和相关噪声生成考试分数
exam1 = base_ability + correlated_noise[:, 0]
exam2 = base_ability + correlated_noise[:, 1]

# 添加一些随机波动和离群值
outlier_mask = np.random.choice([True, False], size=n_samples, p=[0.05, 0.95])
exam1[outlier_mask] = np.random.uniform(0.3, 1.0, size=outlier_mask.sum())
exam2[outlier_mask] = np.random.uniform(0.3, 1.0, size=outlier_mask.sum())

# 分数转换到0-100范围并保留一位小数
exam1 = np.round(exam1 * 100, 1)
exam2 = np.round(exam2 * 100, 1)

# 应用最大值限制(防止超过100分)
exam1 = np.clip(exam1, 0, 100)
exam2 = np.clip(exam2, 0, 100)

# 创建更真实的通过/不通过标签
# 结合两门考试的综合表现，添加一些随机因素模拟意外情况
combined_score = 0.6 * exam1 + 0.4 * exam2
pass_threshold = 60  # 及格线
random_factor = np.random.normal(0, 5, size=n_samples)  # 随机因素
pass_probability = 1 / (1 + np.exp(-(combined_score - pass_threshold + random_factor) / 10))
pass_label = np.random.binomial(1, pass_probability)

# 创建DataFrame
df = pd.DataFrame({
    'exam1': exam1,
    'exam2': exam2,
    'pass': pass_label
})

# # 随机添加一些缺失值模拟未参加考试的情况
# missing_mask = np.random.choice([True, False], size=n_samples, p=[0.03, 0.97])
# df.loc[missing_mask, 'exam1'] = np.nan
# missing_mask = np.random.choice([True, False], size=n_samples, p=[0.03, 0.97])
# df.loc[missing_mask, 'exam2'] = np.nan

excel_path = 'student_exam_data.xlsx'
df.to_excel(excel_path, index=False)
print(f"已生成Excel文件：{excel_path}")
print("可在 Jupyter 中用'pd.read_excel({excel_path})读取")


已生成Excel文件：student_exam_data.xlsx
可在 Jupyter 中用'pd.read_excel({excel_path})读取
